In [41]:
import pandas as pd

In [42]:
import os

Address_df = pd.read_csv('testdata/address.csv')
ContactPoint_df = pd.read_csv('testdata/ContactPoint.csv')
HealthcareOrganization_df = pd.read_csv('testdata/HealthcareOrganization.csv')
HealthcarePersonnel_df = pd.read_csv('testdata/HealthcarePersonnel.csv')
Person_df = pd.read_csv('testdata/Person.csv')
ServiceDepartment_df = pd.read_csv('testdata/ServiceDepartment.csv')

# Specify the encoding to handle the decoding error
golden_record = pd.read_csv('golden_standard_duplicates.csv', encoding='latin1')

In [37]:


# contact_point = ContactPoint_df[ContactPoint_df["identifier"] == "23a7711a-8133-4876-b7eb-dcd9e87a1613"]
# language = contact_point['availableLanguage']
# language = language.str.strip("[]").str.split(',').str[0]
# print(language)

# language = language.iloc[0].strip("'")
# print(language)

# # Strip the single quotes from the language string
# language = language.strip("'")
# print(language)

In [43]:
golden_record

,original_id,duplicate_id,entity_type,variation_type,field_name,original_value,varied_value
0,929c8ac4-bf28-40fb-a8cb-2f6eee5e256e,afb21b96-2c0b-4458-ae16-b0b6320470bd,Address,city_typo,city,Hantumeruitburen,Hnatumeruitburen
1,5ab5d4c1-01a9-4f82-a339-931759f6b809,5e7f55e3-02fa-445b-b233-b968282a0526,Address,house_number_suffix,text,Sennasteeg 120,Sennasteeg 120C
2,6e695b3f-4a17-4d31-934c-cba77dc7b782,e125cf46-2d37-4f32-8daf-5a7b64878e34,Address,city_typo,city,Eggenburg,Eggenurg
3,4d1aa07e-aca2-4647-87fa-b649e2631aac,e34b36ec-a3db-4755-aca7-5393c5a7742f,Address,city_typo,city,East Anneburgh,East Anneurgh
4,a2c1e1d4-131b-4956-872a-5ac130c97d34,c72d4e9d-1fd5-47da-b637-a606c626a46a,Address,postal_format,postalCode,1419 XK,1419XK
...,...,...,...,...,...,...,...
784,0b5560b6-31ce-4544-9707-f76cef137c97,85f437a6-b29e-489c-93c2-f7bb92c7d6c4,ContactPoint,email_domain_change,email,LopezKey.Midwifery@dept.healthcare.org,LopezKey.Midwifery@dept.healthcare.de
785,9d2e5daa-3788-47ce-bcab-7a91183947ef,7e352362-61a7-421f-bbe8-9d642cca4a98,ContactPoint,email_typo,email,Carter.Anesthesia@dept.healthcare.org,Cartr.Anesthesia@dept.healthcare.org
786,09237dcc-b77c-4af1-b5fa-5ae843ff4aa7,eb4066fb-b363-4780-8456-ec0183d47adb,ContactPoint,email_domain_change,email,PerezBecker.Gynecologic@dept.healthcare.org,PerezBecker.Gynecologic@dept.healthcare.de
787,19a14af2-15e0-41f2-907f-88de0fe8b3f2,59eec7fc-1953-416a-b9bb-3039d5f4767d,ContactPoint,email_typo,email,Obrien.Nursing@dept.healthcare.org,Obrien.Nuersing@dept.healthcare.org


In [44]:
# Dictionary to store all dataframes by entity type
entity_dataframes = {}

# Get a static list of variable names first to avoid the iteration error
variable_names = list(locals().keys())

# Populate the dictionary with your existing dataframes
for var_name in variable_names:
    if var_name.endswith('_df'):
        entity_type = var_name.replace('_df', '')
        entity_dataframes[entity_type] = locals()[var_name]

# Iterate through each row in the golden record
for _, row in golden_record.iterrows():
    entity_type = row['entity_type']
    
    # Check if we have a dataframe for this entity type
    if entity_type in entity_dataframes:
        df = entity_dataframes[entity_type]
        
        # Find the matching row using original_id
        mask = df['identifier'] == row['original_id']
        
        # If a match is found, update the specified field with the varied value
        if mask.any():
            field_name = row['field_name']
            if field_name in df.columns:
                df.loc[mask, field_name] = row['varied_value']
                print(f"Updated {entity_type} - {row['original_id']} - {field_name}: {row['varied_value']}")
            else:
                print(f"Warning: Field {field_name} not found in {entity_type} dataframe")
        else:
            print(f"Warning: No row with identifier {row['original_id']} found in {entity_type} dataframe")
    else:
        print(f"Warning: No dataframe found for entity type {entity_type}")

Updated Address - 929c8ac4-bf28-40fb-a8cb-2f6eee5e256e - city: Hnatumeruitburen
Updated Address - 5ab5d4c1-01a9-4f82-a339-931759f6b809 - text: Sennasteeg 120C
Updated Address - 6e695b3f-4a17-4d31-934c-cba77dc7b782 - city: Eggenurg
Updated Address - 4d1aa07e-aca2-4647-87fa-b649e2631aac - city: East Anneurgh
Updated Address - a2c1e1d4-131b-4956-872a-5ac130c97d34 - postalCode: 1419XK
Updated Address - 35951baf-7432-4cbc-983a-f1862d6efa7d - city: LoÃ«bnga
Updated Address - d3b564b0-8be0-4c3e-9c94-938160c6b3ed - postalCode: 4294 
Updated Address - 022c88c0-b563-446c-9cbf-0c64c039c71e - city: Lizn
Updated Address - e90a2364-3a03-473e-bb9b-df46f5930c38 - text: ReischlstraÃe 16A
Updated Address - 02a03b46-f8b8-4e14-a7d8-ed18ce4e682f - text: Dana-Neumayr-StraÃe 111B
Updated Address - 53475d8a-44ba-4427-9e49-dc9e29cf0d42 - country: Netherlands
Updated Address - 1c7bc6c8-d62f-4e8c-abca-09663eef04eb - postalCode: 83054 
Updated Address - 265c2480-2234-4048-9c0e-54d7c307fd76 - country: Netherland

In [45]:
HealthcareOrganization_df

,identifier,healthcareOrganizationName,address,contactPoint
0,71545a13-7a1d-4006-8d72-3104f77383c1,Faulkner-Howard Zorg,c8a70639-eb11-47b3-a7a9-c3787c65c1e5,23a7711a-8133-4876-b7eb-dcd9e87a1613
1,6288e1a5-cc45-4821-98a6-416d1775336d,"Garrett, Salazar and Clayton Tervisekeskus",d3290a4c-b5d3-4b16-a619-4cb1d71037d1,b421eaeb-5340-47ca-baf3-897a3e70f16a
2,5f3f5638-3870-4a14-b490-b6081dfc8352,"Green, Jones and Shelton Tervisekeskus",9cdeb3e6-0870-415c-afcd-81b5d24bace4,09e469e6-ec62-42c8-a648-ee38e07405eb
3,a59cec98-126c-4c8f-b888-447911ebcd49,"Robinson, Montoya and Walters Gesundheitszentrum",daf66c5f-2577-4ffa-887a-746319c16a0d,12f175ff-ae3b-46ec-9a27-d85888c132ad
4,8b5885ca-0bb2-43f0-bd30-291a55fea08e,"Mcneil, Gonzalez and Howard Tervisekeskus",19d5f970-98b3-4c6e-8a14-b90a7795e986,eb21a3f6-e6fd-48e8-969c-91c278601602
5,870f084c-7244-4536-a85e-25b4b3969057,"Nash, Anderson and Collins Zorg",971c702d-5bf4-4c04-ac64-2b4c49b25ded,14aa451c-a69c-4b85-9432-f8db6a174c1c
6,ccc14d51-73f6-40d8-a9f4-1cc04653a560,Loez-Key Gesundheitszentrum,0cc36d8c-7786-4fe5-9675-ebf74fe30c9a,b9bdee2d-d663-449d-955e-18b1fa83ada4
7,fb5eb866-2640-411e-a9f2-c3c74505f4f6,Williams and Sons Gesundheitszentrum,4745dd9e-2789-4389-9f32-77fd1d77ce40,f24dfdd8-5091-4bdc-8ef0-66d44279b14d
8,27d99a23-e4f7-425e-afe6-790abc18a40b,Pock LLC Zorg,bb0378eb-7a62-422e-9d69-d9fc4b1cb8bd,122411e6-ba89-42dd-85e6-9ea9db66bfda
9,f2ad985f-ff3e-4ba1-8ac7-28b4a41865bf,Brady Inc Tervisekeskus,15ace7a1-ceca-4ee3-90da-8a9516408169,8e751eb7-64d0-4913-991b-8adf0202861c


In [46]:
from faker import Faker

fake = Faker()

# Create a copy to avoid modifying original data
A_copy = Address_df.copy()
CP_copy = ContactPoint_df.copy()
P_copy = Person_df.copy()
HO_copy = HealthcareOrganization_df.copy()
SD_copy = ServiceDepartment_df.copy()
HP_copy = HealthcarePersonnel_df.copy()

# Save old values
old_valuesHO = {
    'identifier': HO_copy['identifier'].copy(),
    'address': HO_copy['address'].copy(),
    'contactPoint': HO_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field
HO_copy['identifier'] = HO_copy['identifier'].apply(lambda x: fake.uuid4())
HO_copy['address'] = HO_copy['address'].apply(lambda x: fake.uuid4())
HO_copy['contactPoint'] = HO_copy['contactPoint'].apply(lambda x: fake.uuid4())

# Create copies of dataframes
old_valuesSD = {
    'identifier': SD_copy['identifier'].copy(),
    'address': SD_copy['address'].copy(),
    'contactPoint': SD_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field in ServiceDepartment_df
SD_copy['identifier'] = SD_copy['identifier'].apply(lambda x: fake.uuid4())
SD_copy['address'] = SD_copy['address'].apply(lambda x: fake.uuid4())
SD_copy['contactPoint'] = SD_copy['contactPoint'].apply(lambda x: fake.uuid4())

old_valuesHP = {
    'identifier': HP_copy['identifier'].copy()
}
# Generate new UUIDs for Healthcare Personnel identifiers
HP_copy['identifier'] = HP_copy['identifier'].apply(lambda x: fake.uuid4())

In [47]:
HO_copy

,identifier,healthcareOrganizationName,address,contactPoint
0,d3352057-bbda-4b05-8c76-622b89973416,Faulkner-Howard Zorg,161c51da-16db-4bdf-9ae0-9ff633d2efb1,1a499e5a-edf1-46e5-9db4-af65575735b8
1,655cabcd-64d6-4c82-88c6-737e0efe3a5a,"Garrett, Salazar and Clayton Tervisekeskus",ce138481-8902-4028-b356-3d22d846f3da,69058d69-75ce-4913-8ec8-3c036300c4f0
2,810792dd-3b06-4dd6-baf2-7f5e5f24b901,"Green, Jones and Shelton Tervisekeskus",716d9414-0803-4b4a-9645-6e4ca8d7e826,8b613ca9-1926-4c43-9ca0-ee329cd37960
3,07a12d79-2cb6-48dd-ac5b-a6dbacff7951,"Robinson, Montoya and Walters Gesundheitszentrum",7c40024d-39da-4710-8f0e-77ff563c196b,1b6fe81b-6d45-4db5-89bf-160641ac66e0
4,1395b0a1-a666-4812-b332-632ec39f96f2,"Mcneil, Gonzalez and Howard Tervisekeskus",a314ddb5-1b32-4696-bd11-91ba524c195d,429642a9-0ddf-41d5-b054-4323d639e4af
5,bc4aadc8-6bdd-45d7-b6aa-1ebd28e534d0,"Nash, Anderson and Collins Zorg",70eacd71-536a-4b19-b345-08dd4d3938fe,6cc2162c-f1e0-4947-a9ec-f8fdf3dcb332
6,0b3a120f-e8c3-4f9b-9fae-3dd52983762e,Loez-Key Gesundheitszentrum,b6230b37-981e-4691-b9bd-ca80e2905c7a,dfa0f384-2770-44b5-96e9-8fdd5331d397
7,215bd565-c6d1-4d33-8b36-3dd1be5f56bf,Williams and Sons Gesundheitszentrum,e74cd0cc-70fc-4857-ba17-724a4f9f7760,0a1a98ee-43f6-4c58-a16c-c708eeaa277e
8,379fce57-f677-4343-8f24-803fe04b7864,Pock LLC Zorg,0499dc41-9d95-4c48-bb5e-0c9da1d1777d,3e137f74-948f-40b1-bc38-8088fd5b1a5e
9,4d372cc0-64f9-450c-a9ec-d6bfb7d50893,Brady Inc Tervisekeskus,2c83764b-579d-4ad6-a2da-13c5d66eb094,a06a38c3-ea1c-4e4f-9f2f-50395e069d84


In [48]:
# Create a mapping of old to new UUIDs
HO_uuid_mapping = {
    old_valuesHO['identifier'].iloc[i]: HO_copy['identifier'].iloc[i]
    for i in range(len(HO_copy))
}

# Update references in other dataframes (using the copies)
# Update institution references in Healthcare_Personnel_df copy
HP_copy['institution'] = HP_copy['institution'].map(HO_uuid_mapping).fillna(HP_copy['institution'])

# Update isPartOf references in ServiceDepartment_df copy
SD_copy['isPartOf'] = SD_copy['isPartOf'].map(HO_uuid_mapping).fillna(SD_copy['isPartOf'])

# Update contact point references
ho_contact_mapping = {
    old_valuesHO['contactPoint'].iloc[i]: HO_copy['contactPoint'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in ContactPoint_df copy where it matches old contact points
CP_copy['identifier'] = CP_copy['identifier'].map(ho_contact_mapping).fillna(CP_copy['identifier'])

# Update address references
ho_address_mapping = {
    old_valuesHO['address'].iloc[i]: HO_copy['address'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in Address_df copy where it matches old addresses
A_copy['identifier'] = A_copy['identifier'].map(ho_address_mapping).fillna(A_copy['identifier'])

# Create mapping for ServiceDepartment
sd_uuid_mapping = {
    old_valuesSD['identifier'].iloc[i]: SD_copy['identifier'].iloc[i]
    for i in range(len(SD_copy))
}

# Create address mapping for ServiceDepartment
sd_address_mapping = {
    old_valuesSD['address'].iloc[i]: SD_copy['address'].iloc[i]
    for i in range(len(SD_copy))
}

# Create contact point mapping for ServiceDepartment
sd_contact_mapping = {
    old_valuesSD['contactPoint'].iloc[i]: SD_copy['contactPoint'].iloc[i]
    for i in range(len(SD_copy))
}

# Update references in Address_df copy for ServiceDepartment
A_copy['identifier'] = A_copy['identifier'].map(sd_address_mapping).fillna(A_copy['identifier'])

# Update references in ContactPoint_df copy for ServiceDepartment
CP_copy['identifier'] = CP_copy['identifier'].map(sd_contact_mapping).fillna(CP_copy['identifier'])

# Create mapping for Personnel
hp_uuid_mapping = {
    old_valuesHP['identifier'].iloc[i]: HP_copy['identifier'].iloc[i]
    for i in range(len(HP_copy))
}

# Update references in Person_df copy
P_copy['identifier'] = P_copy['identifier'].map(hp_uuid_mapping).fillna(P_copy['identifier'])

In [49]:
old_valuesHO

{'identifier': 0     71545a13-7a1d-4006-8d72-3104f77383c1
 1     6288e1a5-cc45-4821-98a6-416d1775336d
 2     5f3f5638-3870-4a14-b490-b6081dfc8352
 3     a59cec98-126c-4c8f-b888-447911ebcd49
 4     8b5885ca-0bb2-43f0-bd30-291a55fea08e
 5     870f084c-7244-4536-a85e-25b4b3969057
 6     ccc14d51-73f6-40d8-a9f4-1cc04653a560
 7     fb5eb866-2640-411e-a9f2-c3c74505f4f6
 8     27d99a23-e4f7-425e-afe6-790abc18a40b
 9     f2ad985f-ff3e-4ba1-8ac7-28b4a41865bf
 10    c2472fd6-03e9-4a02-8cea-2df00a66dc4e
 11    f164f9d8-4312-4ce2-9c21-51e17e56ac3d
 12    52ebdac5-a145-4899-a1f8-c1569e0df45b
 13    06d2ed7c-e6ac-4d8a-8160-ff927c7550f2
 14    586f1721-0785-48d7-b118-2d235bf80676
 15    4ec985ff-94b2-4b9d-8881-9f421a42b629
 16    788c161e-f3cc-4d8a-8b16-78e45f20f406
 17    c058a332-b4cf-4fa8-ac98-f73d27fe1b4b
 18    48622a67-4a29-4067-9c66-a27b6a325333
 19    71a8c9c6-0f6a-475b-b55b-2e5ca6ed0ac0
 20    862f78e1-25c2-4b6a-b575-bc6eeee84bcb
 21    f46860a2-a604-45af-b4db-1c1eaa66b464
 22    0b5aafef-85

In [50]:
import os

# Create the new directory if it doesn't exist
if not os.path.exists('testdata_dup'):
    os.makedirs('testdata_dup')

# Save each dataframe to CSV in the new directory
A_copy.to_csv('testdata_dup/address_d.csv', index=False)
CP_copy.to_csv('testdata_dup/ContactPoint_d.csv', index=False)
HO_copy.to_csv('testdata_dup/HealthcareOrganization_d.csv', index=False)
HP_copy.to_csv('testdata_dup/HealthcarePersonnel_d.csv', index=False)
P_copy.to_csv('testdata_dup/Person_d.csv', index=False)
SD_copy.to_csv('testdata_dup/ServiceDepartment_d.csv', index=False)

In [54]:
import pandas as pd
import os

# Define paths
original_data_path = 'testdata'
duplicate_data_path = 'testdata_dup'
golden_standard_path = 'golden_standard_duplicates.csv'
output_path = 'updated_golden_standard_duplicates.csv'

# Read golden standard file
golden_df = pd.read_csv(golden_standard_path, encoding='latin1')

# Dictionary to store matching field values between original and duplicate datasets
field_value_to_id_mapping = {}

# Process each entity type
for _, row in golden_df.iterrows():
    entity_type = row['entity_type']
    field_name = row['field_name']
    varied_value = row['varied_value']
    
    # Define file paths based on entity type
    if entity_type == 'Address':
        original_file = 'address.csv'
        duplicate_file = 'address_d.csv'
    elif entity_type == 'ContactPoint':
        original_file = 'ContactPoint.csv'
        duplicate_file = 'ContactPoint_d.csv'
    elif entity_type == 'HealthcareOrganization':
        original_file = 'HealthcareOrganization.csv'
        duplicate_file = 'HealthcareOrganization_d.csv'
    elif entity_type == 'HealthcarePersonnel':
        original_file = 'HealthcarePersonnel.csv'
        duplicate_file = 'HealthcarePersonnel_d.csv'
    elif entity_type == 'Person':
        original_file = 'Person.csv'
        duplicate_file = 'Person_d.csv'
    elif entity_type == 'ServiceDepartment':
        original_file = 'ServiceDepartment.csv'
        duplicate_file = 'ServiceDepartment_d.csv'
    else:
        continue
    
    # Read duplicate dataset
    duplicate_df = pd.read_csv(os.path.join(duplicate_data_path, duplicate_file))
    
    # Look for records in the duplicate dataset that have the varied value
    # in the specified field
    if field_name in duplicate_df.columns:
        matching_rows = duplicate_df[duplicate_df[field_name] == varied_value]
        
        if not matching_rows.empty:
            # Get the identifier from the first matching row
            new_duplicate_id = matching_rows.iloc[0]['identifier']
            
            # Store the mapping for this specific row
            key = (entity_type, field_name, varied_value)
            field_value_to_id_mapping[key] = new_duplicate_id

# Create a new dataframe for the updated golden standard
updated_golden_df = golden_df.copy()

# Update the duplicate_id column based on our mappings
for i, row in updated_golden_df.iterrows():
    key = (row['entity_type'], row['field_name'], row['varied_value'])
    if key in field_value_to_id_mapping:
        updated_golden_df.at[i, 'duplicate_id'] = field_value_to_id_mapping[key]

# Save the updated golden standard file
updated_golden_df.to_csv(output_path, index=False, encoding='utf-8')

print(f"Updated golden standard file saved to {output_path}")
print(f"Total field-value mappings created: {len(field_value_to_id_mapping)}")
print(f"Duplicate IDs updated: {sum(updated_golden_df['duplicate_id'] != golden_df['duplicate_id'])}")

Updated golden standard file saved to updated_golden_standard_duplicates.csv
Total field-value mappings created: 703
Duplicate IDs updated: 789
